# Order data cleanup (datacamp project)

As a Data Engineer at an electronics e-commerce company, Voltmart, you have been requested by a peer Machine Learning team to clean the data containing the information about orders made last year. They are planning to further use this cleaned data to build a demand forecasting model. To achieve this, they have shared their requirements regarding the desired output table format.

An analyst shared a parquet file called `"orders_data.parquet"` for you to clean and preprocess. 

You can see the dataset schema below along with the **cleaning requirements**:

## `orders_data.parquet`

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `order_date` | `timestamp` | Date and time when the order was made | _Modify: Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date_ |
| `time_of_day` | `string` | Period of the day when the order was made | _New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm_ |
| `order_id` | `long` | Order ID | _N/A_ |
| `product` | `string` | Name of a product ordered | _Remove rows containing "TV" as the company has stopped selling this product; ensure all values are lowercase_ |
| `product_ean` | `double` | Product ID | _N/A_ |
| `category` | `string` | Broader category of a product | _Ensure all values are lowercase_ |
| `purchase_address` | `string` | Address line where the order was made ("House Street, City, State Zipcode") | _N/A_ |
| `purchase_state` | `string` | US State of the purchase address | _New column containing: the State that the purchase was ordered from_ |
| `quantity_ordered` | `long` | Number of product units ordered | _N/A_ |
| `price_each` | `double` | Price of a product unit | _N/A_ |
| `cost_price` | `double` | Cost of production per product unit | _N/A_ |
| `turnover` | `double` | Total amount paid for a product (quantity x price) | _N/A_ |
| `margin` | `double` | Profit made by selling a product (turnover - cost) | _N/A_ |

<br>

In [1]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

25/03/18 00:55:51 WARN Utils: Your hostname, dmtr-t14 resolves to a loopback address: 127.0.1.1; using 192.168.178.10 instead (on interface wlp3s0)
25/03/18 00:55:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 00:55:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/18 00:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [32]:
orders_data = spark.read.parquet('data/orders_data.parquet')
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


# Solution

## time_of_day + order_date

New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm 

Remove orders placed between 12am and 5am, convert from timestamp to date

In [33]:
# time of day column
orders_data = orders_data.withColumn("time_of_day", 
        F.when( ( F.hour(F.col('order_date')) >= 5) & ( F.hour(F.col('order_date')) < 12), 'morning' ) \
        .when( ( F.hour(F.col('order_date')) >= 12) & ( F.hour(F.col('order_date')) < 18), 'afternoon' ) \
        .when( ( F.hour(F.col('order_date')) >= 18) & ( F.hour(F.col('order_date')) < 24), 'evening' )
        .otherwise( None ))
# filter out Nones -> made between midnight and 5am
orders_data = orders_data.filter(~orders_data['time_of_day'].isNull())
orders_data_df = orders_data.toPandas()
print(orders_data_df['time_of_day'].unique()) # verify the unique values and removed midnight to 5am

['evening' 'afternoon' 'morning']


,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning


In [36]:
print(orders_data.dtypes)
orders_data = orders_data.withColumn("order_date", orders_data["order_date"].cast(types.DateType()))
print(orders_data.dtypes) # verify the casted column

[('order_date', 'timestamp_ntz'), ('order_id', 'bigint'), ('product', 'string'), ('product_id', 'double'), ('category', 'string'), ('purchase_address', 'string'), ('quantity_ordered', 'bigint'), ('price_each', 'double'), ('cost_price', 'double'), ('turnover', 'double'), ('margin', 'double'), ('time_of_day', 'string')]
[('order_date', 'date'), ('order_id', 'bigint'), ('product', 'string'), ('product_id', 'double'), ('category', 'string'), ('purchase_address', 'string'), ('quantity_ordered', 'bigint'), ('price_each', 'double'), ('cost_price', 'double'), ('turnover', 'double'), ('margin', 'double'), ('time_of_day', 'string')]


## category
all values lowercase

In [ ]:
orders_data = orders_data.withColumn("category", F.lower(orders_data["category"])) # lower case the category column

## product
remove all values containing TV, ensure lowercase

In [ ]:
orders_data = orders_data.withColumn("product", F.lower(orders_data["product"])) # lower case the product column
orders_data = orders_data.filter(~orders_data['product'].contains("tv")) # filter out rows with 'tv' in the product column

## purchase_state
New column containing: the State that the purchase was ordered from

In [43]:
orders_data = orders_data.withColumn("purchase_state", F.substring(F.col("purchase_address"), -8,2))
orders_data.toPandas().purchase_state.unique() # verify that only states were extracted

array(['MA', 'OR', 'CA', 'TX', 'WA', 'GA', 'NY', 'ME'], dtype=object)

## inspect cleaned up data

In [10]:
orders_data.collect()
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day,purchase_state
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening,MA
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon,OR
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon,CA
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening,CA
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning,TX


## save the updated file as `orders_data_clean.parquet`

In [44]:
orders_data.write.parquet('data/orders_data_clean.parquet', mode='overwrite')

# verify it worked

In [45]:
orders_data = spark.read.parquet('data/orders_data_clean.parquet')
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day,purchase_state
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening,MA
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon,OR
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon,CA
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening,CA
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning,TX
